# Image classifiers training
## 1. Data Augmentation

In [8]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

train_dir = './train_images'
test_dir = './test_images'

# Define the transformations for augmentation
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(), 
        transforms.RandomResizedCrop(36),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0,),std=(1,))
    ]),
    'test':transforms.Compose([
        transforms.Grayscale(), 
        transforms.ToTensor(), 
        transforms.Normalize(mean=(0,),std=(1,))
    ])        
}
train_data = torchvision.datasets.ImageFolder(train_dir, transform=data_transforms['train'])
test_data = torchvision.datasets.ImageFolder(test_dir, transform=data_transforms['test'])


In [16]:
len(train_data)

91720

In [17]:
len(test_data)

7628

In [18]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
valid_size = 0.2
batch_size = 32

num_train = len(train_data)
indices_train = list(range(num_train))
np.random.shuffle(indices_train)
split_tv = int(np.floor(valid_size * num_train))
train_new_idx, valid_idx = indices_train[split_tv:],indices_train[:split_tv]

train_sampler = SubsetRandomSampler(train_new_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=5)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=5)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=5)
classes = ('noface','face')

In [12]:
import torch.nn as nn
import torch.nn.functional as F


class Net_3_layers(nn.Module):
    def __init__(self):
        super(Net_3_layers, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 6 * 6, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

path = "models/3-layer-net.pth"
model = Net_3_layers().to(device)
torch.save(model.state_dict(), path)

In [13]:
class Net_4_layers(nn.Module):
    def __init__(self):
        super(Net_4_layers, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 6 * 6, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16) # Add a new fully connected layer
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
path = "models/4-layer-net.pth"
model_1 = Net_4_layers().to(device)
torch.save(model_1.state_dict(), path)

In [11]:
def train_NN(n_epochs, optimizer, model, criterion):
    for epoch in range(1, n_epochs+1):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

    print('Finished Training')
    

In [21]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer_Adam = optim.Adam(model_1.parameters(), lr=0.001, weight_decay=1e-4) # Use Adam optimization
optimizer_SGD = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
n_epochs = 20

#train_NN(n_epochs, optimizer_SGD, model, criterion)
train_NN(n_epochs, optimizer_Adam, model_1, criterion)



[2,  2000] loss: 0.391
[3,  2000] loss: 0.272
[4,  2000] loss: 0.220
[5,  2000] loss: 0.187
[6,  2000] loss: 0.164
[7,  2000] loss: 0.153
[8,  2000] loss: 0.144
[9,  2000] loss: 0.134
[10,  2000] loss: 0.128
[11,  2000] loss: 0.127
[12,  2000] loss: 0.121
[13,  2000] loss: 0.117
[14,  2000] loss: 0.115
[15,  2000] loss: 0.110
[16,  2000] loss: 0.110
[17,  2000] loss: 0.105
[18,  2000] loss: 0.105
[19,  2000] loss: 0.105
[20,  2000] loss: 0.102
[21,  2000] loss: 0.102
Finished Training


In [22]:
def get_classifier_accuracy(model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [23]:
model_accuracy = get_classifier_accuracy(model)
model_1_accuracy = get_classifier_accuracy(model_1)
print('Accuracy of the 3-layer classifier on the 7628 test images: {:.2f}'.format(model_accuracy))
print('Accuracy of the 4-layer classifier on the 7628 test images: {:.2f}'.format(model_1_accuracy))

Accuracy of the 3-layer classifier on the 7628 test images: 98.03
Accuracy of the 4-layer classifier on the 7628 test images: 98.30


**Observation**
By applying data augmentation techniques on the fly, we obtained a classifier of 98% accuracy, which is 6% higher than the one trained with original data.
Then we added a fully-connected layer (64,32) and replaced SGD with Adam as optimization function. The accuracy of image classifier increased by 0.3%.

## 2. Transfer Learning
Since RESNET18 was trained on RGB images, we do not apply GrayScale transform in pre-processing step.

In [24]:
# Define the transformations for augmentation
data_transforms_RGB = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(36),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test':transforms.Compose([
        transforms.ToTensor(), 
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])        
}
train_data_RGB = torchvision.datasets.ImageFolder(train_dir, transform=data_transforms_RGB['train'])
test_data_RGB = torchvision.datasets.ImageFolder(test_dir, transform=data_transforms_RGB['test'])


In [25]:
train_loader = torch.utils.data.DataLoader(train_data_RGB, batch_size=batch_size, sampler=train_sampler, num_workers=5)
valid_loader = torch.utils.data.DataLoader(train_data_RGB, batch_size=batch_size, sampler=valid_sampler, num_workers=5)
test_loader = torch.utils.data.DataLoader(test_data_RGB, batch_size=batch_size, shuffle=True, num_workers=5)

In [26]:
import torch.nn as nn
model_ft = torchvision.models.resnet18(weights='IMAGENET1K_V1')
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)
model_2 = model_ft.to(device)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer_SGD = optim.SGD(model_2.parameters(), lr=0.001, momentum=0.9)
n_epochs = 20

train_NN(n_epochs, optimizer_SGD, model_2, criterion)
